# MMT Projekt
**Relativität von Farben**

## Imports
Importiert die für das Programm notwendigen Pakete.
customtkinter muss installiert werden, der rest ist bestandteil von Python.

- *customtkinter*: stellt die Funktionalität für eine Grafische Benutzeroberfläche zur Verfügung
- *time*: ermöglicht eine clock zu bausen, die für den Effekt genutzt wird indem der Hintergrund regelmäßig die Farbe ändert
- *colorsys*: vereinfacht die umrechnung von Farbcodierungen, welche für die Information für den Nutzer gebraucht wird
- *enum*:    reduziert redundanzen für die Farben der UI-Elemente

In [1]:
import customtkinter 
import time          
import colorsys      
from enum import auto

# weil es einfach besser aussieht.
customtkinter.set_appearance_mode("dark")

## Redundanzvermeidung für das Hauptprogramm

In [2]:
class COLOR(auto):
        """COLOR
        Stellt sicher, dass die Farben des UI's konsistent sind
        ohen ein externes Design Dokument einbinden zu müssen
        """
        uiColor = "#E86E4D"
        hoverColor = "#e24920"
        borderColor = "#372e29"
        progressColor="#f8d3c9"
        textColor = "gray"
        disabledButtonColor = "#555555"
        disabledProgressColor ="#999999"

In [3]:
class SliderFrame(customtkinter.CTkFrame):
    def __init__(self,parent,name, start, end, value, callback, tooltip=None, onLeave=None, enabled=True):
        """
        parent (CTkBaseclass): Das Element in dem die Textbox verankert werden soll
        label (Str): die Überschrift der Textbox (notwendig)
        text (str): der anzuzeigende Text in der Box (notwendig)

        w (Int): Breite des Frames
        h (int): höhe des Frames

        border (int): breite des Rahmens
        """
        super().__init__(parent)
        
        self.configure(
            width=330,
            height=20,
            fg_color="transparent",
            border_width=0,
            border_color=COLOR.borderColor
            )
  
        self.Slider =None

        if(enabled): self.Slider = customtkinter.CTkSlider(
                master = self,
                from_=start, 
                to=end,
                button_color=COLOR.uiColor,
                button_hover_color=COLOR.hoverColor,
                progress_color=COLOR.progressColor,
                hover=True,
                command=callback)  
        else: self.Slider = customtkinter.CTkSlider(
                master = self,
                from_=start, 
                to=end,
                button_color=COLOR.disabledButtonColor,
                progress_color=COLOR.disabledProgressColor,
                state="disabled",
                command=callback)
        self.Slider.set(value)

        self.Label = customtkinter.CTkLabel(
            master= self,
            text=name,
            height=20,
            text_color=COLOR.textColor,
            bg_color="transparent")
        self.Label.cget("font").configure(size=12)
        
        self.Value = customtkinter.CTkLabel(
            master=self,
            text= str(value),
            height=20,
            text_color=COLOR.textColor,
            bg_color="transparent")
        self.Value.cget("font").configure(size=12)
        
        self.Label.place(relx=.275, rely=.5, anchor=customtkinter.E)
        self.Value.place(relx=.355, rely=.5, anchor=customtkinter.E)
        self.Slider.place(relx=.38, rely=.5, anchor=customtkinter.W)

        self.bind("<Enter>", tooltip)
        self.Label.bind("<Enter>", tooltip)
        self.Value.bind("<Enter>", tooltip)
        self.Slider.bind("<Enter>", tooltip)

        self.bind("<Leave>", onLeave)
        self.Label.bind("<Leave>", onLeave)
        self.Value.bind("<Leave>", onLeave)
        self.Slider.bind("<Leave>", onLeave)

In [4]:
class TextBox(customtkinter.CTkFrame):
    """TextBox
    Eine Klasse, die einen Frame mit Textbox und Label einheitlich erzeugt, um den Code der eigentlichen Anwendung verhältnismäßig sauber zu halten.   
    """
    def __init__(self,parent,label,text,w=500, h=500, border=0):
        """
        parent (CTkBaseclass): Das Element in dem die Textbox verankert werden soll
        label (Str): die Überschrift der Textbox (notwendig)
        text (str): der anzuzeigende Text in der Box (notwendig)

        w (Int): Breite des Frames
        h (int): höhe des Frames

        border (int): breite des Rahmens
        """
        super().__init__(parent)
        
        self.configure(
            width=w,
            height=h,
            fg_color="transparent",
            border_width=border,
            border_color=COLOR.borderColor
            )

        self.textBox = customtkinter.CTkTextbox(self,width=w-(w*0.05), height=h-(h*0.1)-25, border_spacing=20)
        self.textBox.cget("font").configure(size=16)
        self.textBox.insert("0.0", text)  # insert at line 0 character 0
        self.textBox.configure(state="disabled")  # configure textbox to be read-only

        self.Label = customtkinter.CTkLabel(
            master=self,
            text=label,
            height=22,
            text_color=COLOR.uiColor,
            bg_color="transparent")
        self.Label.cget("font").configure(size=32, weight="bold")
        
        self.textBox.place(relx=.5,rely=0.975, anchor= customtkinter.S)
        self.Label.place(relx=.025,rely=0.025, anchor= customtkinter.NW)



In [5]:
class InfoWindow(customtkinter.CTkToplevel):
    """InfoWindow
    
    Ein Fenster welches dem Nutzer Informationen zum Programm und der Anwendung anbietet.
    Das Fenster besteht aus einem Button, der das Fenster wieder schlißt und einer Textbox mit den hier beschriebenen Informationen.

    Die Auslagerung dient vorrangig dem Zweck den Code übersichtlich zu halten.

    hat 
    """
    def __init__(self, parent):
        """Nimmt als Argument das aufrufende Element"""
        super().__init__(parent)

        self.attributes('-fullscreen',True)
        self.title("Beschreibung")
        self.parent = parent

        self.aboutText = """Diese Programm soll die Relativität einer Farbe im Kontext ihrer Umgebung
zeigen. Dafür eignet sich die Farbe Braun besonders, weil es kein braunes Licht 
gibt. Braun wird dann wahrgenommen, wenn orangenes Licht relativ zu seiner Umwelt dunkler ist.

In diesem Programm kann die Sättigung und die Helligkeit einer Orange-
abstufung sowie die Helligkeit der Umgebung angepasst werden um diesen 
Effekt zu verdeutlichen.

Verschieben sie die Slider in den Farbeinstellungen oder klicken Sie auf den 
Playbutton [ ▶ ] um den Effekt zu erforschen.

Am besten wird der Effekt deutlich, wenn man den Gradienten aktiviert,
den Wert der Geschwindigkeit klein hält und sich auf die Form in der Mitte des Bildschirms konzentriert.

Mit dem [ ✕ ]-Button oben rechts in der Ecke verlassen Sie das Programm. 
"""

        self.aboutFrame = TextBox(self, label="Illusion", text = self.aboutText, w=650, h=500, border=0)

        self.back = customtkinter.CTkButton(self.aboutFrame,
                text='Okay',
                fg_color=COLOR.uiColor,
                hover_color=COLOR.hoverColor,
                command=self.backCallback)

        self.aboutFrame.place(relx=.5,rely=0.5,anchor=customtkinter.CENTER)
        self.back.place(relx=.5,rely=1,anchor=customtkinter.S)
        
    def backCallback(self):
        """
        Schließt das Fenster und offenbart das Hauptfenster
        """
        self.parent.deiconify()
        self.destroy()

In [6]:
class WarningWindow(customtkinter.CTkToplevel):
    """WarningWindow
    
    Ein Fenster welches dem Nutzer über das Risiko der Automatischen Bildwechsel aufklärt und eine Epilepsiewarnung gibt.
    Erst wenn der Nutzer das Risiko anerkannt hat wird die Funktionalität des Playbuttons in der Hauptanwendung aktiv.

    Es besteht aus 
    - einer Textbox mit der Warnung, 
    - einer Checkbox für die aktive Kenntnisnahme der Warnung
    - einem Okay Button und einem Zurück Button

    der Okay button schaltet die Funktionalität frei und schließt das Fenster und offenbart das Hauptfenster
    der Zurückbutton schließt das Fenster und offenbart das Hauptfenster

    Die Auslagerung dient vorrangig dem Zweck den Code übersichtlich zu halten.

    hat 
    """
    def __init__(self, parent):
        super().__init__(parent)
        self.attributes('-fullscreen',True)
        self.title("Epilepsiewarnung")

        self.check_var = "off"
        self.parent = parent
        self.text="""Sie sind im Begriff eine Funktion aufzurufen, welche die Hintergrundfarbe des
Bildschirms in regelmäßigen Zeitabständen (0.5 bis 2sek) von schwarz zu weiß wechseln lässt.
    
Manche Menschen zeigen bei bestimmten Arten von flimmerndern Lichtquellen
oder ähnlichen, in der Umgebung täglich vorkommenden Elementen eine
Neigung zu epileptischen Krisen oder Ohnmachtsanfällen.
Dieses Risiko kann bei bestimmten Fernsehbildern oder Videospielen auftreten. Das Phänomen kann auch auftreten, ohne dass die betreffende Person in
diesem Zusammenhang je medizinisch behandelt wurde oder einen 
epileptischen Anfall erlitten hat.

Sollten Sie selbst oder ein Mitglied Ihrer Familie je bei flimmernden Licht-
quellen epilepsie-ähnliche Symptome empfunden haben, so verzichten Sie bitte auf die Nutzung dieses Features. 

Brechen Sie das Programm sofort ab, und konsultieren Sie einen Arzt, falls Sie eines der folgenden Symptome verspüren: 
   
    - Schwindel
    - Sehstörungen
    - Augen- oder Muskelkontraktionen
    - Bewußtseinsverlust
    - Orientierungschwierigkeiten
    - unkontrollierte Bewegungen oder krampfartige Zuckungen
    
Wenn Sie erneut auf den Play-Button drücken, startet Die Funktion"""
        
        self.textBox = TextBox(self, "EPILEPSIE WARNUNG", self.text, 650, 775)

        self.okay = customtkinter.CTkButton(
            self.textBox, 
            text="Okay",
            fg_color=COLOR.disabledButtonColor,
            hover_color=COLOR.hoverColor,
            command=self.okCallback)
        self.okay.configure(state="disabled")

        self.cancel = customtkinter.CTkButton(
            self.textBox,
            text="Zurück",
            fg_color=COLOR.uiColor,
            hover_color=COLOR.hoverColor,
            command=self.cancelCallback)
        
        self.checkbox = customtkinter.CTkCheckBox(
            master=self.textBox,
            text="Ich bin mir des Risikos bewusst.", 
            command=self.checkBoxCallback,
            onvalue="on", 
            offvalue="off",
            hover_color=COLOR.uiColor,
            border_color=COLOR.uiColor,
            fg_color=COLOR.hoverColor)
        self.checkbox.deselect()
        
        self.checkbox.place(relx=0.5,rely=0.9, anchor=customtkinter.S)
        self.textBox.place(relx=0.5,rely=0.5,anchor=customtkinter.CENTER)
        self.cancel.place(relx=0.6,rely=0.975, anchor=customtkinter.W)
        self.okay.place(relx=0.4,rely=0.975, anchor=customtkinter.E)

    def checkBoxCallback(self):
        """
        aktiviert den Okay button
        dient der aktiven kenntnisnahme der Warnung
        """
        if(self.checkbox.get() == "on"):
            self.okay.configure(state="normal",fg_color=COLOR.uiColor,hover_color=COLOR.hoverColor)
        else:
            self.okay.configure(state="disabled",fg_color=COLOR.disabledButtonColor)
    
    def okCallback(self):
        """
        schaltet die Funktionalitäten für die automatische Farbänderung im Hauptfenster frei
        schließt das Fenster und offenbart das Hauptfenster

        """
        self.parent.warningNoticed = True
        self.parent.gradientSwitch.configure(state="normal", button_color=COLOR.uiColor, button_hover_color=COLOR.hoverColor, progress_color=COLOR.progressColor)
        self.parent.time_slider.Slider.configure(state="normal", button_color=COLOR.uiColor, button_hover_color=COLOR.hoverColor, progress_color=COLOR.progressColor)
        self.parent.deiconify()
        self.destroy()
        
    def cancelCallback(self):
        """
        schließt das Fenster und offenbart das Hauptfenster
        """
        self.parent.deiconify()
        self.destroy()

## Hauptanwendung


In [7]:

class App(customtkinter.CTk):
    """App
    Erzeugt eine grafische Benutzeroberfläche 
    """
    def __init__(self):
        super().__init__()
        #Flags
        self.open = True            # Beendet Mainloop wenn False/schließt Programm
        self.running = False        # Bestimmt ob sich der Hintergrund automatisch verändert
        self.warningNoticed = False # schaltet Zeiteinstellungen Frei wenn True (über WarningWindow)
        
        #Colors
        self.hsv= [13,84,64]        # Box Farbe in HSV
        self.color = "#A3381B"      # Box Farbe in HEX
        self.bgSat = 0              # Hintergrundhelligkeit
        self.bgColor = "#000000"    # Hintergrundfarbe

        #Speed
        self.speed = 1              # Updategeschwindigkeit des Hintergrundwechsels
        self.switch_var = False     # Graduelle änderung des Hintergrundes wenn True

        self.WarningWindow = None   # Platzhalter für WarningWindow
        self.InfoWindow = None      # Platzhalter für InfoWindow
        self.authors = "Henry Lorenz & Fabian Lampe"

        self.title("MMT Project")
        self.attributes('-fullscreen',True)     # blendet Fensterrahmen aus um maximale Fläche mit der Hintergrundfarbe zu nutzen
        self.config(bg = self.bgColor)          # setzt initial die Hintergrundfarbe
        
        #Elements
        # closeButton erstellt den Butten zum beenden des Programms (durch Fullscreen verlore geangen)
        self.closeButton = customtkinter.CTkButton(
            master=self,
            text="✕",
            width=20, height=20, 
            command=self.closeCallback,
            fg_color=COLOR.uiColor, hover_color=COLOR.hoverColor)
        # / closeButton

        # Orange Box
        self.orangeBox = customtkinter.CTkFrame(
            master=self,
            width=200, height=200, corner_radius=50,
            bg_color=self.bgColor, fg_color=self.color)
        # /Orange Box

        # Authorlabel
        self.authorsLabel = customtkinter.CTkLabel(
            master=self,
            text= " Autoren: " + str(self.authors),
            height=20,
            bg_color=self.bgColor, text_color=COLOR.textColor)
        self.authorsLabel.cget("font").configure(size=12)
        # /Authorlabel

        # settingsFrame
        self.settingsFrame = customtkinter.CTkFrame(
            master=self,
            width=350, height=330, border_width=1,
            bg_color=self.bgColor, border_color=COLOR.borderColor)

        self.settingsLabel = customtkinter.CTkLabel(
            master=self.settingsFrame,
            text="Einstellungen ⚙",
            height=20,
            text_color=COLOR.textColor,bg_color="transparent")
        self.settingsLabel.cget("font").configure(size=16,weight="bold")

        # color Frame
        self.hsvFrame = customtkinter.CTkFrame(
            master=self.settingsFrame,
            width=335, height=150, border_width=1,
            fg_color="transparent",border_color=COLOR.borderColor)

        self.hsvLabel = customtkinter.CTkLabel(
            master=self.hsvFrame,
            text="Farbe",
            height=22,
            text_color=COLOR.textColor, bg_color="transparent")
        self.hsvLabel.cget("font").configure(size=14, slant="italic")

        self.hexLabel = customtkinter.CTkLabel(
            master=self.hsvFrame,
            text=str(self.rgb2hex(self.hsv2rgb(self.hsv))),
            height=22,
            text_color=COLOR.uiColor, bg_color="transparent")
        self.hexLabel.cget("font").configure(size=16, weight="bold")
        self.hexLabel.bind("<Enter>", lambda x: self.ToolTip.configure(text="HEX Farbcode der Farbbox"))
        self.hexLabel.bind("<Leave>", self.on_leave)
        #/ colorfrane

        # speed Frame
        self.speedFrame = customtkinter.CTkFrame(
            master=self.settingsFrame,
            width=335,height=100,
            border_width=1,
            fg_color="transparent",border_color=COLOR.borderColor)
        
        self.speedLabel = customtkinter.CTkLabel(
            master=self.speedFrame,
            text="Zeit",
            height=22,
            text_color=COLOR.textColor, bg_color="transparent")
        self.speedLabel.cget("font").configure(size=14, slant="italic")
        # /speedframe

        #Tooltip
        self.ToolTip = customtkinter.CTkLabel(
            master=self.settingsFrame,
            text="",
            width=340, height=40,
            bg_color="transparent", text_color=COLOR.textColor)
        self.ToolTip.cget("font").configure(size=12)
        self.ToolTip.place(rely=1,relx=.5, anchor=customtkinter.S)

        #/Tooltip

        # color Inputs

        self.h_slider = SliderFrame(self.hsvFrame,"Value", 0, 360, self.hsv[0], 
        None,
        lambda x: self.ToolTip.configure(text="Konstanter Hue Wert der Farbe (hsv)"), self.on_leave)
        self.h_slider.Slider.destroy()
        
        self.s_slider = SliderFrame(self.hsvFrame,"Saturation", 30, 100, self.hsv[1], 
        self.s_sliderCallback,
        lambda x: self.ToolTip.configure(text="Sättigungs Wert der Farbe (hsv)"), self.on_leave)
        
        self.v_slider = SliderFrame(self.hsvFrame,"Value", 10, 100, self.hsv[2], 
        self.v_sliderCallback ,
        lambda x: self.ToolTip.configure(text="Helligkeit der Farbe (hsv)"), self.on_leave)

        self.bg_slider = SliderFrame(self.hsvFrame, "BG Saturation", 0, 100, self.bgSat,
        self.bg_sliderCallback,
        lambda x: self.ToolTip.configure(text="Helligkeit der Hintergrundfarbe"),self.on_leave)


        self.hexLabel.place(relx=0.95,rely=0.1,anchor=customtkinter.NE)
        self.h_slider.place(relx=.005, rely=.35, anchor=customtkinter.SW)
        self.s_slider.place(relx=.005, rely=.575, anchor=customtkinter.SW)
        self.v_slider.place(relx=.005, rely=.775, anchor=customtkinter.SW)
        self.bg_slider.place(relx=.005, rely=.975, anchor=customtkinter.SW)

        # /color inputs

        self.gradientSwitch = customtkinter.CTkSwitch(
            master=self.speedFrame, text="Gradient",
            command=self.gradientSwitchCallback,
            onvalue="on", 
            offvalue="off",
            button_color=COLOR.disabledButtonColor,
            progress_color=COLOR.progressColor,
            text_color=COLOR.textColor,
            state="disabled",
            button_hover_color=COLOR.hoverColor)
        self.gradientSwitch.bind("<Enter>", lambda x: self.ToolTip.configure(text="Hintergrundfarbe wechselt über Graustufen"))
        self.gradientSwitch.bind("<Leave>", self.on_leave)

        # time Inputs
        #self.time_slider = self.buildSlider(self.speedFrame, "Geschwindigkeit", 0.5, 2, self.speed, self.time_sliderCallback, 
        #lambda x: self.ToolTip.configure(text="Geschwindigkeit des Hintergrundfarbwechsels (sek)"), False)

        self.time_slider = SliderFrame(self.speedFrame, "Geschwindigkeit", 0.5, 2, self.speed,
        self.time_sliderCallback, 
        lambda x: self.ToolTip.configure(text="Geschwindigkeit des Hintergrundfarbwechsels (sek)"),self.on_leave, False)

        self.time_button =customtkinter.CTkButton(
            master= self.speedFrame,
            text="▶",
            width=90,
            height=35, 
            command=self.pauseCallback,
            fg_color=COLOR.uiColor,
            hover_color=COLOR.hoverColor
        )
        self.time_button.bind("<Enter>", lambda x: self.ToolTip.configure(text="Startet/Stoppt automatische Farbänderung \nder Hintergrundfarbe"))
        self.time_button.bind("<Leave>", self.on_leave)
 
        self.info_button =customtkinter.CTkButton(
            master= self.speedFrame,
            text="❓",
            width=35,
            height=35, 
            command=self.infoCallback,
            fg_color=COLOR.uiColor,
            hover_color=COLOR.hoverColor
        )
        self.info_button.cget("font").configure(size=20, weight="bold")
        self.info_button.bind("<Enter>", lambda x: self.ToolTip.configure(text="Informationen zur Illusion und zum Program"))
        self.info_button.bind("<Leave>", self.on_leave)

        self.time_slider.place(relx=.005, rely=.45, anchor=customtkinter.SW)
        
        self.time_button.place(relx=.34,rely=0.92,anchor=customtkinter.SE)
        self.info_button.place(relx=.96,rely=0.92,anchor=customtkinter.SE)
        self.gradientSwitch.place(relx=0.39,rely=0.85,anchor=customtkinter.SW)
        # /time Inputs

        # /settingsFrame

        #Place Elements
        #general Elements
        self.closeButton.place(relx = 1, rely = 0,anchor=customtkinter.NE)
        self.closeButton.bind("<Enter>", 
        lambda x: self.ToolTip.configure(text="Beendet das Programm"))
        self.closeButton.bind("<Leave>", self.on_leave)

        self.authorsLabel.place(relx = 0, rely = 1, anchor=customtkinter.SW)
        self.orangeBox.place(relx=0.5, rely=0.5, anchor=customtkinter.CENTER)

        #hsvFrame Elements
        self.hsvFrame.place(relx=.02,rely=.1,anchor=customtkinter.NW)
        self.hsvLabel.place(relx=.01, rely=.01, anchor=customtkinter.NW)

        self.speedFrame.place(relx=.02,rely=.88,anchor=customtkinter.SW)
        self.speedLabel.place(relx=.01, rely=.01, anchor=customtkinter.NW)


        #Setting Elements
        self.settingsFrame.place(relx=1, rely=1, anchor=customtkinter.SE)
        self.settingsLabel.place(relx=.01, rely=.01, anchor=customtkinter.NW)

# Callback Methodes

    def on_leave(self, enter):
        """
        löscht den Text aus dem Tooltiplabel
        """
        self.ToolTip.configure(text="")

    def closeCallback(self):
        """
        beendet Mainloop durch Flag 'open' und beendet das Programm
        """
        self.open = False
        self.destroy()
    
    def gradientSwitchCallback(self):
        """
        setzt Flag für den graduellen Übergang in der Hintergrundanimation
        """
        self.switch_var = not self.switch_var

    def s_sliderCallback(self,value):
        """
        Beobachtet den aufrufenden Slider und 
            - aktualisiert das Label des Sliders mit dem neuen wert
            - aktualisiert die Farbe der Box, den HEX und den HSV Code der Farbe
        """
        self.hsv[1] = value
        self.s_slider.Value.configure(text="{0:>2}".format(str(round(self.hsv[1]))))
        self.color=str(self.rgb2hex(self.hsv2rgb(self.hsv)))
        self.hexLabel.configure(text="{}".format(self.color))
        self.orangeBox.configure(fg_color=self.color)
    
    def v_sliderCallback(self,value):
        """
        Beobachtet den aufrufenden Slider und 
            - aktualisiert das Label des Sliders mit dem neuen wert
            - aktualisiert die Farbe der Box, den HEX und den HSV Code der Farbe in der Klasse
        """
        self.hsv[2] = value
        self.v_slider.Value.configure(text="{0:>2}".format(str(round(self.hsv[2]))))
        self.color=str(self.rgb2hex(self.hsv2rgb(self.hsv)))
        self.hexLabel.configure(text="{}".format(self.color))
        self.orangeBox.configure(fg_color=self.color)

    def time_sliderCallback(self,value):
        """
        Beobachtet den aufrufenden Slider und 
            - aktualisiert das Label des Sliders mit dem neuen wert
            - die speed variable der Klasse
        """
        self.speed = value
        self.time_slider.Value.configure(text="{0:>2}".format(str(round(self.speed,1))))

    def bg_sliderCallback(self,value):
        """
        Beobachtet den aufrufenden Slider und 
            - aktualisiert das Label des Sliders mit dem neuen wert
            - die hintergrundhelligkeit der Klasse
            - updatet die Hintergrundfarbe entsprechend
        """
        self.bgSat = value
        self.bg_slider.Value.configure(text="{0:>2}".format(str(round(self.bgSat))))
        self.bgColor = self.rgb2hex(self.hsv2rgb([0,0,self.bgSat]))
        self.config(bg=self.bgColor)
    
    def pauseCallback(self):
        """
        wenn die Epilepsiewarnung noch nicht akzeptiert wurde, wird WarningWindow erzeugt
        sonst wird die running Flag gesetzt, welche die while-loop für das regelmäßige ändern der Hintergrundfarbe startet/stoppt
        Als indikator für den Nutzer wird das Icon auf dem button zwischen "Play" und "Pause" gewechselt
        und die Manuelle Manipulation der Hintergrundhelligkeit unterbunden solange running True ist
        """
        if(self.warningNoticed == False): self.create_toplevel()
        else:
            if(self.running):
                self.running = False
                self.time_button.configure(text="▶")
                self.bg_slider.Slider.configure(state = "normal", button_color=COLOR.uiColor, button_hover_color=COLOR.hoverColor, progress_color=COLOR.progressColor)
            else:
                self.running = True
                self.time_button.configure(text="⏸")
                self.bg_slider.Slider.configure(state ="disabled", button_color=COLOR.disabledButtonColor,progress_color=COLOR.disabledProgressColor)  


# Layout Methodes
    def create_toplevel(self):
       self.WarningWindow = WarningWindow(self)
       self.withdraw()
    
    def infoCallback(self):
        self.InfoWindow = InfoWindow(self)
        self.withdraw()
       
# Utility Methods
    def rgb2hex(self,rgb):
            r,g,b = round(rgb[0]),round(rgb[1]),round(rgb[2])
            val = "#{:02X}{:02X}{:02X}".format(r,g,b)
            return val

    def hex2rgb(self,hex):
            h = hex.strip("#") 
            rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
            return rgb
        
    def rgb2hsv(self,rgb):
            r,g,b = (rgb[0]/255),(rgb[1]/255),(rgb[2]/255)
            # Constraining the values to the range 0 to 1
            # defining the following terms for convenience
            Cmax = max(r, g, b)
            Cmin = min(r, g, b)
            delta = Cmax - Cmin
            # hue calculation
            if (delta == 0):  H = 0
            elif (Cmax == r): H = (60 * (((g  - b) / delta) % 6))
            elif (Cmax == g): H = (60 * (((b  - r) / delta) + 2))
            elif (Cmax == b): H = (60 * (((r  - g) / delta) + 4))
            
            # saturation calculation
            if (Cmax == 0): S = 0 
            else: S = delta / Cmax
            # value calculation
            V = Cmax 
            return(round(H),round(S*100),round(V*100))

    def hsv2rgb(self,hsv):
            h,s,v = (hsv[0]/360),(hsv[1]/100),(hsv[2]/100)
            rgb= colorsys.hsv_to_rgb(h,s,v)
            return (round(rgb[0]*255),round(rgb[1]*255),round(rgb[2]*255))

In [8]:
if __name__ == "__main__":
    app = App()
    time_1 = 0
    switch=True

    while 1:
        time_2 = time.process_time()
        app.update_idletasks()
        app.update()
        delta = time_2 - time_1        
 
        if app.running and app.switch_var:
            if(delta >= app.speed):switch = not switch
            if switch and delta<app.speed:
                    app.bgSat=100-round((delta*100)/app.speed)
                    app.bgColor = app.rgb2hex(app.hsv2rgb([0,0,app.bgSat]))
                    app.config(bg=app.bgColor)
                    app.bg_slider.Slider.set(app.bgSat)
                    app.bg_slider.Value.configure(text="{0:>2}".format(str(round(app.bgSat))))
            elif (not switch) and (delta<app.speed):
                    app.bgSat=round((delta*100)/app.speed)
                    app.bgColor = app.rgb2hex(app.hsv2rgb([0,0,app.bgSat]))
                    app.config(bg=app.bgColor)
                    app.bg_slider.Slider.set(app.bgSat)
                    app.bg_slider.Value.configure(text="{0:>2}".format(str(round(app.bgSat))))
            
            
        elif app.running and not app.switch_var and delta > app.speed:
            if app.cget("background") == "black":
                app.config(bg = "white")
                app.orangeBox.configure(bg_color="white")
            else:
                app.config(bg = "black")
                app.orangeBox.configure(bg_color="black")
        
        if delta > app.speed: time_1 = time_2
        if(app.open == False):
            break

bgerror failed to handle background error.
    Original error: invalid command name "140186123894464check_dpi_scaling"
    Error in bgerror: can't invoke "tk" command: application has been destroyed
bgerror failed to handle background error.
    Original error: invalid command name "140186259466112update"
    Error in bgerror: can't invoke "tk" command: application has been destroyed


TclError: invalid command name ".!ctkframe2"